In [1]:
import pandas as pd
import nltk
import numpy as np

In [48]:
df = pd.read_csv(r"F:\Amazon reviews\Reviews.csv")

In [6]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [8]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [49]:
def sentiment(x):
    if x<3:
        return "positive"
    else:
        return "negative"

In [16]:
df.shape

(568454, 10)

In [50]:
df_removed = df[df['Score']!=3]
df = df_removed
df.shape

(525814, 10)

In [51]:
df["Score"] = df["Score"].map(sentiment)
#df["Score"] = df["Score"].apply(lambda x:"positive" if x>3 else "negative")

In [53]:
df = df.drop_duplicates(subset = {"UserId","ProfileName","Time","Text"})
final_df = df[df['HelpfulnessNumerator']<=df["HelpfulnessDenominator"]] 
final_df.shape

(364171, 10)

In [66]:
X = final_df["Text"].head(100000)
print(X.shape)
y = final_df["Score"].head(100000)

(100000,)


In [67]:
from nltk.corpus import stopwords           
from nltk.stem import PorterStemmer

In [68]:
stop = set(stopwords.words('english')) 
import re
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in X:
    sentence = sentence.lower()                
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)    
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence) 
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]
    temp.append(words)
    
X = temp 

In [72]:
print(X[:2])

[['bought', 'sever', 'vital', 'can', 'dog', 'food', 'product', 'found', 'good', 'qualiti', 'product', 'look', 'like', 'stew', 'process', 'meat', 'smell', 'better', 'labrador', 'finicki', 'appreci', 'product', 'better'], ['product', 'arriv', 'label', 'jumbo', 'salt', 'peanut', 'peanut', 'actual', 'small', 'size', 'unsalt', 'sure', 'error', 'vendor', 'intend', 'repres', 'product', 'jumbo']]


In [73]:
sent = []
for row in X:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

X = sent
print(X[:2])

[' bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better', ' product arriv label jumbo salt peanut peanut actual small size unsalt sure error vendor intend repres product jumbo']


In [83]:
print(np.unique(y,return_counts=True))

(array(['negative', 'positive'], dtype=object), array([83899, 16101], dtype=int64))


In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer         

In [119]:
final_tf = X
tf_idf = TfidfVectorizer(ngram_range = (1,3))
tf_data = tf_idf.fit_transform(final_tf)
print(tf_data[1])

  (0, 3115849)	0.07011653073586861
  (0, 247991)	0.06078651148202399
  (0, 2190151)	0.07212772466086945
  (0, 2135427)	0.2471687820637165
  (0, 3403623)	0.06248803428841047
  (0, 2930915)	0.13402913140610523
  (0, 76755)	0.05688031392899929
  (0, 3605806)	0.05574242335508055
  (0, 3586057)	0.05752105130870878
  (0, 4227877)	0.11136715097859334
  (0, 3854865)	0.05591161012269494
  (0, 1311743)	0.10703676258534266
  (0, 4295751)	0.09009125413696044
  (0, 2076971)	0.0932105464357536
  (0, 3314597)	0.10977111311434166
  (0, 3117499)	0.09498619033075846
  (0, 249478)	0.15355684131869216
  (0, 2191381)	0.16646120616482413
  (0, 2135469)	0.16646120616482413
  (0, 3406972)	0.13551711101192365
  (0, 2932892)	0.13632208807930948
  (0, 2930946)	0.15355684131869216
  (0, 81222)	0.1424527876708418
  (0, 3611035)	0.10372945448087148
  (0, 3591766)	0.16646120616482413
  (0, 4228016)	0.16646120616482413
  (0, 3856469)	0.16646120616482413
  (0, 1311965)	0.16646120616482413
  (0, 4296033)	0.166461206164

In [124]:
print(tf_data.shape)

(100000, 4538920)


In [125]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test=train_test_split(tf_data,y,test_size=0.3,random_state=0)

C:\Users\IITS\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [126]:
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression()
lg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [127]:
pred=lg.predict(x_test)
score=lg.score(x_test,y_test)
score2=lg.score(x_train,y_train)
print(score,score2)

0.8433333333333334 0.8427714285714286


In [128]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [129]:
pred=RFC.predict(x_test)
score=RFC.score(x_test,y_test)
score2=RFC.score(x_train,y_train)
print(score,score2)

0.8422 0.9643142857142857


In [134]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [137]:
score = clf.score(x_train,y_train)
score2 = clf.score(x_test,y_test)
print(score,score2)

0.8427714285714286 0.8433333333333334
